# 🛡️ 第二章 练习 3: 双重稳健估计 (Doubly Robust Estimation)

---

## 「双保险」的智慧

在前两个练习中，我们学习了：
- **PSM/IPW**: 依赖倾向得分模型 $e(X) = P(T=1|X)$
- **回归调整**: 依赖结果模型 $\mu(X) = E[Y|X, T]$

但现实中，我们怎么知道模型是否正确呢？

> 「如果倾向得分模型错了，IPW 就完蛋了...」
> 
> 「如果结果模型错了，回归调整也完蛋了...」

有没有一种方法，可以像买保险一样，即使一个模型错了也没关系？

**有的！这就是双重稳健 (Doubly Robust) 估计！**

### 双重稳健的魔力 ✨

双重稳健估计器有一个神奇的性质：

> **只要倾向得分模型或结果模型之一正确，估计就是无偏的！**

这就像买了「双保险」——两道防线，任一道有效就行！

---

## 📚 本节学习目标

1. 理解双重稳健性的核心概念
2. 实现 AIPW (Augmented IPW) 估计器
3. 验证双重稳健性质
4. 对比不同因果推断方法
5. 理解交叉拟合 (Cross-fitting)

In [ ]:
# 导入必要的库
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression, Ridge, LinearRegression
from sklearn.model_selection import KFold
from typing import Tuple, Dict

# 设置
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'SimHei']
plt.rcParams['axes.unicode_minus'] = False
sns.set_style('whitegrid')

print("✅ 环境准备完毕！")

---

## 🌟 Part 1: AIPW 的核心思想

### 回顾：两种基本方法

**方法 1: 回归调整（结果模型）**

$$\hat{ATE}_{\text{reg}} = \frac{1}{n}\sum_{i=1}^{n}[\hat{\mu}_1(X_i) - \hat{\mu}_0(X_i)]$$

其中 $\hat{\mu}_1(X) = \hat{E}[Y|X, T=1]$，$\hat{\mu}_0(X) = \hat{E}[Y|X, T=0]$

**方法 2: IPW（倾向得分模型）**

$$\hat{ATE}_{\text{IPW}} = \frac{1}{n}\sum_{i=1}^{n}\left[\frac{T_i Y_i}{\hat{e}(X_i)} - \frac{(1-T_i) Y_i}{1-\hat{e}(X_i)}\right]$$

### AIPW：两全其美

AIPW (Augmented IPW) 巧妙地结合了两种方法：

$$\hat{ATE}_{\text{AIPW}} = \frac{1}{n}\sum_{i=1}^{n}\left[\underbrace{\hat{\mu}_1(X_i) - \hat{\mu}_0(X_i)}_{\text{回归调整}} + \underbrace{\frac{T_i(Y_i - \hat{\mu}_1(X_i))}{\hat{e}(X_i)}}_{\text{处理组 IPW 修正}} - \underbrace{\frac{(1-T_i)(Y_i - \hat{\mu}_0(X_i))}{1-\hat{e}(X_i)}}_{\text{控制组 IPW 修正}}\right]$$

### 直觉理解

AIPW 的三项分别是什么意思？

| 项 | 公式 | 含义 |
|---|------|------|
| 第一项 | $\hat{\mu}_1(X) - \hat{\mu}_0(X)$ | 结果模型预测的效应 |
| 第二项 | $\frac{T(Y - \hat{\mu}_1(X))}{e(X)}$ | 对处理组预测误差的 IPW 修正 |
| 第三项 | $\frac{(1-T)(Y - \hat{\mu}_0(X))}{1-e(X)}$ | 对控制组预测误差的 IPW 修正 |

**关键洞察**：
- 如果结果模型完美（$Y = \hat{\mu}$），那第二、三项为 0，只剩第一项
- 如果结果模型不完美但倾向得分正确，IPW 修正会纠正偏差
- 所以只要一个模型正确，估计就是无偏的！

---

## 📐 AIPW 双重稳健性证明

**定理**: 如果倾向得分模型或结果模型之一正确，AIPW 估计量是 ATE 的一致估计。

**证明思路**:

AIPW 估计 $E[Y(1)]$ 的部分为：

$$\hat{\mu}_{AIPW,1} = \frac{1}{n}\sum_{i=1}^{n}\left[\hat{\mu}_1(X_i) + \frac{T_i(Y_i - \hat{\mu}_1(X_i))}{\hat{e}(X_i)}\right]$$

我们需要证明：**只要 $\hat{e}$ 或 $\hat{\mu}_1$ 之一正确，$E[\hat{\mu}_{AIPW,1}] = E[Y(1)]$**

---

### 情况 1: 倾向得分模型正确 ($\hat{e} = e$)

$$E\left[\hat{\mu}_1(X) + \frac{T(Y - \hat{\mu}_1(X))}{e(X)}\right]$$

对 X 条件化：

$$= E\left[E\left[\hat{\mu}_1(X) + \frac{T(Y - \hat{\mu}_1(X))}{e(X)} \Big| X\right]\right]$$

$$= E\left[\hat{\mu}_1(X) + \frac{E[T(Y - \hat{\mu}_1(X))|X]}{e(X)}\right]$$

$$= E\left[\hat{\mu}_1(X) + \frac{e(X) \cdot E[Y - \hat{\mu}_1(X)|X,T=1]}{e(X)}\right]$$

$$= E\left[\hat{\mu}_1(X) + E[Y|X,T=1] - \hat{\mu}_1(X)\right]$$

$$= E[E[Y|X,T=1]]$$

应用无混淆假设：

$$= E[Y(1)]$$ ✓

**关键**：即使 $\hat{\mu}_1(X) \neq E[Y|X,T=1]$（结果模型错了），只要倾向得分正确，IPW 修正项会完美抵消偏差！

---

### 情况 2: 结果模型正确 ($\hat{\mu}_1 = \mu_1$)

$$E\left[\mu_1(X) + \frac{T(Y - \mu_1(X))}{e(X)}\right]$$

对 X, T 条件化：

$$= E\left[\mu_1(X)\right] + E\left[\frac{T(Y - \mu_1(X))}{e(X)}\right]$$

对于第二项：

$$E\left[\frac{T(Y - \mu_1(X))}{e(X)}\right] = E\left[E\left[\frac{T(Y - \mu_1(X))}{e(X)} \Big| X\right]\right]$$

由于 $\mu_1(X) = E[Y|X,T=1]$：

$$E[Y - \mu_1(X)|X,T=1] = 0$$

因此第二项为 0：

$$E\left[\mu_1(X) + \frac{T(Y - \mu_1(X))}{e(X)}\right] = E[\mu_1(X)] = E[E[Y|X,T=1]] = E[Y(1)]$$ ✓

**关键**：即使倾向得分模型 $\hat{e}$ 错了，只要结果模型正确，残差的期望为 0，IPW 修正项不产生额外偏差！

---

### 双重保险的魔力

| 倾向得分 $\hat{e}$ | 结果模型 $\hat{\mu}$ | AIPW 估计 |
|---------|---------|------|
| ✅ 正确 | ✅ 正确 | ✅ 无偏，效率最高 |
| ✅ 正确 | ❌ 错误 | ✅ 无偏（IPW 修正救场） |
| ❌ 错误 | ✅ 正确 | ✅ 无偏（残差为 0） |
| ❌ 错误 | ❌ 错误 | ❌ 有偏 |

这就是"双重稳健"的含义：**两道防线，任一道有效就够！**

---

---

## 🔬 Part 2: 实现 AIPW

In [ ]:
def estimate_outcome_models(
    X: np.ndarray,
    T: np.ndarray,
    Y: np.ndarray
) -> Tuple[np.ndarray, np.ndarray]:
    """
    估计结果模型
    
    mu_1(X) = E[Y|X, T=1]
    mu_0(X) = E[Y|X, T=0]
    
    分别为处理组和控制组训练模型，然后对所有样本预测
    """
    treated_mask = T == 1
    control_mask = T == 0
    
    # TODO: 训练处理组的结果模型 mu_1(X)
    model_1 = Ridge(alpha=1.0)
    pass  # 👈 你的代码
    
    # 对所有样本预测
    mu_1 = None  # 👈 你的代码
    
    # TODO: 训练控制组的结果模型 mu_0(X)
    model_0 = Ridge(alpha=1.0)
    pass  # 👈 你的代码
    
    mu_0 = None  # 👈 你的代码
    
    return mu_1, mu_0

<details><summary>💡 点击查看参考答案</summary>

```python
def estimate_outcome_models(
    X: np.ndarray,
    T: np.ndarray,
    Y: np.ndarray
) -> Tuple[np.ndarray, np.ndarray]:
    """
    估计结果模型
    
    mu_1(X) = E[Y|X, T=1]
    mu_0(X) = E[Y|X, T=0]
    
    分别为处理组和控制组训练模型，然后对所有样本预测
    """
    treated_mask = T == 1
    control_mask = T == 0
    
    # 训练处理组的结果模型 mu_1(X)
    model_1 = Ridge(alpha=1.0)
    model_1.fit(X[treated_mask], Y[treated_mask])
    
    # 对所有样本预测
    mu_1 = model_1.predict(X)
    
    # 训练控制组的结果模型 mu_0(X)
    model_0 = Ridge(alpha=1.0)
    model_0.fit(X[control_mask], Y[control_mask])
    
    mu_0 = model_0.predict(X)
    
    return mu_1, mu_0
```

</details>

In [ ]:
def estimate_ate_aipw(
    X: np.ndarray,
    T: np.ndarray,
    Y: np.ndarray,
    propensity: np.ndarray,
    mu_1: np.ndarray,
    mu_0: np.ndarray
) -> Tuple[float, float]:
    """
    使用 AIPW 估计 ATE
    
    AIPW = E[(mu_1 - mu_0) + T*(Y - mu_1)/e - (1-T)*(Y - mu_0)/(1-e)]
    """
    # 裁剪倾向得分
    propensity_clipped = np.clip(propensity, 0.01, 0.99)
    
    # TODO: 计算 AIPW 的三项
    
    # 第一项：结果模型预测的差异
    term1 = None  # 👈 你的代码: mu_1 - mu_0
    
    # 第二项：处理组的 IPW 修正
    term2 = None  # 👈 你的代码: T * (Y - mu_1) / propensity_clipped
    
    # 第三项：控制组的 IPW 修正
    term3 = None  # 👈 你的代码: (1 - T) * (Y - mu_0) / (1 - propensity_clipped)
    
    # TODO: AIPW 得分
    aipw_scores = None  # 👈 你的代码: term1 + term2 - term3
    
    if aipw_scores is None:
        return None, None
    
    # ATE 估计
    ate = aipw_scores.mean()
    
    # 标准误
    se = aipw_scores.std() / np.sqrt(len(Y))
    
    return ate, se

<details><summary>💡 点击查看参考答案</summary>

```python
def estimate_ate_aipw(
    X: np.ndarray,
    T: np.ndarray,
    Y: np.ndarray,
    propensity: np.ndarray,
    mu_1: np.ndarray,
    mu_0: np.ndarray
) -> Tuple[float, float]:
    """
    使用 AIPW 估计 ATE
    
    AIPW = E[(mu_1 - mu_0) + T*(Y - mu_1)/e - (1-T)*(Y - mu_0)/(1-e)]
    """
    # 裁剪倾向得分
    propensity_clipped = np.clip(propensity, 0.01, 0.99)
    
    # 计算 AIPW 的三项
    
    # 第一项：结果模型预测的差异
    term1 = mu_1 - mu_0
    
    # 第二项：处理组的 IPW 修正
    term2 = T * (Y - mu_1) / propensity_clipped
    
    # 第三项：控制组的 IPW 修正
    term3 = (1 - T) * (Y - mu_0) / (1 - propensity_clipped)
    
    # AIPW 得分
    aipw_scores = term1 + term2 - term3
    
    # ATE 估计
    ate = aipw_scores.mean()
    
    # 标准误
    se = aipw_scores.std() / np.sqrt(len(Y))
    
    return ate, se
```

</details>

In [ ]:
# 测试 AIPW
np.random.seed(42)
n = 2000

# 生成数据
X = np.random.randn(n, 3)
propensity_logit = 1.5 * (X[:, 0] + 0.5 * X[:, 1])
propensity_true = 1 / (1 + np.exp(-propensity_logit))
T = np.random.binomial(1, propensity_true)
true_ate = 2.0
Y = 5 + true_ate * T + 1.5 * X[:, 0] + X[:, 1] + np.random.randn(n) * 0.5

print(f"📊 数据概览:")
print(f"   样本量: {n}")
print(f"   真实 ATE: {true_ate}")
print(f"   处理组: {T.sum()} ({T.mean()*100:.1f}%)")

# 朴素估计
naive_ate = Y[T==1].mean() - Y[T==0].mean()
print(f"   朴素 ATE: {naive_ate:.4f} (偏差: {naive_ate - true_ate:+.4f})")

In [ ]:
# 估计倾向得分
lr = LogisticRegression(max_iter=1000)
lr.fit(X, T)
propensity = lr.predict_proba(X)[:, 1]

print(f"倾向得分范围: [{propensity.min():.4f}, {propensity.max():.4f}]")

# 估计结果模型
mu_1, mu_0 = estimate_outcome_models(X, T, Y)

if mu_1 is not None and mu_0 is not None:
    print(f"\n结果模型:")
    print(f"   mu_1 范围: [{mu_1.min():.2f}, {mu_1.max():.2f}]")
    print(f"   mu_0 范围: [{mu_0.min():.2f}, {mu_0.max():.2f}]")
    print(f"   预测的平均效应: {(mu_1 - mu_0).mean():.4f}")
    
    # AIPW 估计
    aipw_ate, aipw_se = estimate_ate_aipw(X, T, Y, propensity, mu_1, mu_0)
    
    if aipw_ate is not None:
        print(f"\n📈 AIPW 估计:")
        print(f"   AIPW ATE: {aipw_ate:.4f} ± {aipw_se:.4f}")
        print(f"   95% CI: [{aipw_ate - 1.96*aipw_se:.4f}, {aipw_ate + 1.96*aipw_se:.4f}]")
        print(f"   偏差: {aipw_ate - true_ate:+.4f}")
    else:
        print("❌ 请完成 estimate_ate_aipw 函数！")
else:
    print("❌ 请完成 estimate_outcome_models 函数！")

---

## 🧪 Part 3: 验证双重稳健性

这是本节最重要的实验！我们要验证 AIPW 的「双保险」性质。

### 实验设计

| 场景 | 倾向得分模型 | 结果模型 | 预期 |
|-----|-------------|---------|------|
| 1 | ✅ 正确 | ✅ 正确 | 无偏 |
| 2 | ✅ 正确 | ❌ 错误 | 无偏 |
| 3 | ❌ 错误 | ✅ 正确 | 无偏 |
| 4 | ❌ 错误 | ❌ 错误 | 有偏 |

In [ ]:
def simulate_with_misspecification(
    n: int = 3000,
    propensity_correct: bool = True,
    outcome_correct: bool = True,
    seed: int = 42
):
    """
    生成数据，可以人为引入模型误设定
    
    真实 DGP 包含非线性项：
    - logit(e) = 1.5 * (X1 + 0.5*X2 + 0.3*X1^2)
    - Y = 5 + 2*T + 1.5*X1 + X2 + 0.5*X1^2 + noise
    
    如果只用线性模型（不包含 X1^2），就是误设定
    """
    np.random.seed(seed)
    
    # 生成基础特征
    X1 = np.random.randn(n)
    X2 = np.random.randn(n)
    X3 = np.random.randn(n)
    
    # 真实倾向得分（包含非线性）
    propensity_logit = 1.5 * (X1 + 0.5*X2 + 0.3*X1**2)
    propensity_true = 1 / (1 + np.exp(-propensity_logit))
    T = np.random.binomial(1, propensity_true)
    
    # 真实结果（包含非线性）
    true_ate = 2.0
    Y = 5 + true_ate*T + 1.5*X1 + X2 + 0.5*X1**2 + np.random.randn(n)*0.5
    
    # 根据正确性返回不同的特征
    if propensity_correct and outcome_correct:
        # 两个都正确：包含非线性特征
        X = np.column_stack([X1, X2, X3, X1**2])
    elif propensity_correct and not outcome_correct:
        # 只有倾向得分正确
        X_prop = np.column_stack([X1, X2, X3, X1**2])
        X_outcome = np.column_stack([X1, X2, X3])  # 缺少 X1^2
        return X_prop, X_outcome, T, Y, true_ate
    elif not propensity_correct and outcome_correct:
        # 只有结果模型正确
        X_prop = np.column_stack([X1, X2, X3])  # 缺少 X1^2
        X_outcome = np.column_stack([X1, X2, X3, X1**2])
        return X_prop, X_outcome, T, Y, true_ate
    else:
        # 两个都错误：缺少非线性特征
        X = np.column_stack([X1, X2, X3])
    
    return X, X, T, Y, true_ate

In [ ]:
def test_double_robustness(n: int = 3000) -> pd.DataFrame:
    """
    测试双重稳健性质
    
    四种场景的完整测试
    """
    scenarios = [
        ('两模型都正确', True, True),
        ('只有倾向得分正确', True, False),
        ('只有结果模型正确', False, True),
        ('两模型都错误', False, False)
    ]
    
    results = []
    
    for name, prop_correct, outcome_correct in scenarios:
        # 生成数据
        data = simulate_with_misspecification(n, prop_correct, outcome_correct)
        
        if len(data) == 5:
            X_prop, X_outcome, T, Y, true_ate = data
        else:
            X, _, T, Y, true_ate = data
            X_prop = X_outcome = X
        
        # 估计倾向得分
        lr = LogisticRegression(max_iter=1000)
        lr.fit(X_prop, T)
        propensity = lr.predict_proba(X_prop)[:, 1]
        
        # 估计结果模型
        mu_1, mu_0 = estimate_outcome_models(X_outcome, T, Y)
        
        if mu_1 is None or mu_0 is None:
            continue
        
        # AIPW 估计
        aipw_ate, aipw_se = estimate_ate_aipw(X_prop, T, Y, propensity, mu_1, mu_0)
        
        if aipw_ate is None:
            continue
        
        bias = aipw_ate - true_ate
        
        results.append({
            '场景': name,
            '倾向得分正确': '✅' if prop_correct else '❌',
            '结果模型正确': '✅' if outcome_correct else '❌',
            'AIPW ATE': aipw_ate,
            'SE': aipw_se,
            '偏差': bias,
            '|偏差|': abs(bias)
        })
    
    return pd.DataFrame(results)

In [ ]:
# 运行双重稳健性测试
dr_results = test_double_robustness(n=3000)

if not dr_results.empty:
    print("🛡️ 双重稳健性验证:")
    print("=" * 80)
    print(f"真实 ATE = 2.0")
    print()
    display(dr_results)
    
    print("\n💡 关键发现:")
    print("   1. 两模型都正确时：偏差最小")
    print("   2. 只有一个模型正确时：偏差仍然很小（双重稳健性！）")
    print("   3. 两模型都错误时：偏差明显增大")
    
    # 可视化
    fig, ax = plt.subplots(figsize=(10, 6))
    
    colors = ['green', 'orange', 'orange', 'red']
    bars = ax.barh(dr_results['场景'], dr_results['|偏差|'], color=colors, alpha=0.7)
    ax.axvline(0.1, color='gray', linestyle='--', label='|偏差| = 0.1')
    ax.set_xlabel('|偏差|', fontsize=12)
    ax.set_title('双重稳健性验证: 不同场景下的偏差', fontsize=14)
    
    for bar, bias in zip(bars, dr_results['偏差']):
        ax.text(bar.get_width() + 0.01, bar.get_y() + bar.get_height()/2,
               f'{bias:+.4f}', va='center', fontsize=10)
    
    plt.tight_layout()
    plt.show()
else:
    print("请先完成前面的 TODO 部分！")

---

## 📊 Part 4: 方法对比

让我们系统地比较四种因果推断方法。

In [ ]:
def compare_causal_methods(
    X: np.ndarray,
    T: np.ndarray,
    Y: np.ndarray,
    true_ate: float
) -> pd.DataFrame:
    """
    对比不同的因果推断方法
    
    1. 朴素估计
    2. 回归调整
    3. IPW
    4. AIPW
    """
    results = []
    
    # 1. 朴素估计
    naive_ate = Y[T==1].mean() - Y[T==0].mean()
    results.append({
        '方法': '朴素估计',
        'ATE': naive_ate,
        '偏差': naive_ate - true_ate,
        'SE': None,
        '依赖': '无'
    })
    
    # 2. 回归调整
    X_with_T = np.column_stack([T, X])
    reg = LinearRegression()
    reg.fit(X_with_T, Y)
    reg_ate = reg.coef_[0]
    results.append({
        '方法': '回归调整',
        'ATE': reg_ate,
        '偏差': reg_ate - true_ate,
        'SE': None,
        '依赖': '结果模型'
    })
    
    # 3. IPW
    lr = LogisticRegression(max_iter=1000)
    lr.fit(X, T)
    propensity = lr.predict_proba(X)[:, 1]
    propensity_clipped = np.clip(propensity, 0.01, 0.99)
    
    weights = T / propensity_clipped + (1-T) / (1 - propensity_clipped)
    y1_ipw = (Y * weights * T).sum() / (weights * T).sum()
    y0_ipw = (Y * weights * (1-T)).sum() / (weights * (1-T)).sum()
    ipw_ate = y1_ipw - y0_ipw
    
    results.append({
        '方法': 'IPW',
        'ATE': ipw_ate,
        '偏差': ipw_ate - true_ate,
        'SE': None,
        '依赖': '倾向得分模型'
    })
    
    # 4. AIPW
    mu_1, mu_0 = estimate_outcome_models(X, T, Y)
    if mu_1 is not None and mu_0 is not None:
        aipw_ate, aipw_se = estimate_ate_aipw(X, T, Y, propensity, mu_1, mu_0)
        if aipw_ate is not None:
            results.append({
                '方法': 'AIPW',
                'ATE': aipw_ate,
                '偏差': aipw_ate - true_ate,
                'SE': aipw_se,
                '依赖': '双重稳健'
            })
    
    return pd.DataFrame(results)

In [ ]:
# 运行方法对比
comparison = compare_causal_methods(X, T, Y, true_ate)

if not comparison.empty:
    print("📊 因果推断方法对比:")
    print("=" * 70)
    print(f"真实 ATE = {true_ate}")
    print()
    display(comparison)
    
    # 可视化
    fig, ax = plt.subplots(figsize=(10, 6))
    
    methods = comparison['方法']
    ates = comparison['ATE']
    colors = ['red', 'orange', 'blue', 'green']
    
    bars = ax.bar(methods, ates, color=colors, alpha=0.7)
    ax.axhline(true_ate, color='black', linestyle='--', linewidth=2, label=f'真实 ATE = {true_ate}')
    ax.set_ylabel('ATE 估计', fontsize=12)
    ax.set_title('不同因果推断方法的 ATE 估计', fontsize=14)
    ax.legend()
    
    for bar, ate, bias in zip(bars, ates, comparison['偏差']):
        ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.05,
               f'偏差: {bias:+.3f}', ha='center', fontsize=10)
    
    plt.tight_layout()
    plt.show()

---

## 🔄 Part 5: 交叉拟合 (Cross-fitting)

### 为什么需要交叉拟合？

当我们使用机器学习模型（如随机森林、神经网络）时，容易出现**过拟合偏差**：
- 模型在训练集上表现太好
- 导致 $Y - \hat{\mu}(X)$ 的残差被低估
- IPW 修正项失效

### 交叉拟合的解决方案

1. 把数据分成 K 折
2. 对每一折：
   - 用其他 K-1 折训练模型
   - 用当前折做预测
3. 汇总所有折的 AIPW 得分

这样模型的训练和预测是在不同数据上进行的，避免了过拟合。

In [ ]:
def cross_fitting_aipw(
    X: np.ndarray,
    T: np.ndarray,
    Y: np.ndarray,
    n_folds: int = 5
) -> Tuple[float, float]:
    """
    使用交叉拟合的 AIPW
    """
    n = len(Y)
    aipw_scores = np.zeros(n)
    
    kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)
    
    for train_idx, test_idx in kf.split(X):
        # TODO: 分割数据
        X_train, X_test = X[train_idx], X[test_idx]
        T_train, T_test = T[train_idx], T[test_idx]
        Y_train, Y_test = Y[train_idx], Y[test_idx]
        
        # TODO: 在训练集上训练倾向得分模型
        lr = LogisticRegression(max_iter=1000)
        lr.fit(X_train, T_train)
        propensity_test = lr.predict_proba(X_test)[:, 1]
        
        # TODO: 在训练集上训练结果模型
        treated_train = T_train == 1
        control_train = T_train == 0
        
        model_1 = Ridge(alpha=1.0)
        model_1.fit(X_train[treated_train], Y_train[treated_train])
        mu_1_test = model_1.predict(X_test)
        
        model_0 = Ridge(alpha=1.0)
        model_0.fit(X_train[control_train], Y_train[control_train])
        mu_0_test = model_0.predict(X_test)
        
        # 计算测试集上的 AIPW 得分
        propensity_clipped = np.clip(propensity_test, 0.01, 0.99)
        
        term1 = mu_1_test - mu_0_test
        term2 = T_test * (Y_test - mu_1_test) / propensity_clipped
        term3 = (1 - T_test) * (Y_test - mu_0_test) / (1 - propensity_clipped)
        
        aipw_scores[test_idx] = term1 + term2 - term3
    
    # 计算 ATE 和标准误
    ate = aipw_scores.mean()
    se = aipw_scores.std() / np.sqrt(n)
    
    return ate, se

In [ ]:
# 对比标准 AIPW 和交叉拟合 AIPW
if mu_1 is not None:
    # 交叉拟合
    cf_ate, cf_se = cross_fitting_aipw(X, T, Y, n_folds=5)
    
    print("🔄 交叉拟合 vs 标准 AIPW:")
    print("=" * 50)
    print(f"真实 ATE: {true_ate}")
    print(f"\n标准 AIPW:")
    print(f"   ATE: {aipw_ate:.4f} ± {aipw_se:.4f}")
    print(f"   偏差: {aipw_ate - true_ate:+.4f}")
    print(f"\n交叉拟合 AIPW:")
    print(f"   ATE: {cf_ate:.4f} ± {cf_se:.4f}")
    print(f"   偏差: {cf_ate - true_ate:+.4f}")
    
    print("\n💡 说明:")
    print("   在使用线性模型时，两者差异不大")
    print("   但在使用复杂机器学习模型时，交叉拟合更稳健")

---

## 📝 Part 6: 思考题

### 问题 1: 什么是双重稳健性？为什么 AIPW 具有这个性质？

**你的答案:**

*（在这里写下你的思考...）*

---

### 问题 2: AIPW 估计器的三项分别代表什么含义？

**你的答案:**

*（在这里写下你的思考...）*

---

### 问题 3: 如果两个模型都错误，AIPW 还会有偏吗？为什么？

**你的答案:**

*（在这里写下你的思考...）*

---

### 问题 4: 什么是交叉拟合？为什么在使用机器学习模型时需要交叉拟合？

**你的答案:**

*（在这里写下你的思考...）*

---

### 问题 5: 双重稳健估计是「银弹」吗？有什么局限性？

**提示:** 想想未观测混淆、模型选择、样本量...

**你的答案:**

*（在这里写下你的思考...）*

---

In [ ]:
class MyAIPWEstimator:
    """
    从零实现的 AIPW (Augmented IPW) 估计器
    
    实现了完整的双重稳健估计流程
    """
    
    def __init__(self, 
                 propensity_model=None,
                 outcome_model=None,
                 trim_propensity: Tuple[float, float] = (0.01, 0.99)):
        """
        Args:
            propensity_model: 倾向得分模型（默认 LogisticRegression）
            outcome_model: 结果模型（默认 Ridge）
            trim_propensity: 倾向得分裁剪范围
        """
        self.propensity_model = propensity_model or LogisticRegression(max_iter=1000)
        self.outcome_model = outcome_model or Ridge(alpha=1.0)
        self.trim_propensity = trim_propensity
        
    def fit(self, X: np.ndarray, T: np.ndarray, Y: np.ndarray):
        """拟合倾向得分模型和结果模型"""
        self.X_ = X
        self.T_ = T
        self.Y_ = Y
        
        # Step 1: 估计倾向得分
        self.propensity_model.fit(X, T)
        self.propensity_ = self.propensity_model.predict_proba(X)[:, 1]
        self.propensity_ = np.clip(self.propensity_, 
                                   self.trim_propensity[0],
                                   self.trim_propensity[1])
        
        # Step 2: 估计结果模型
        treated_mask = T == 1
        control_mask = T == 0
        
        # mu_1(X) = E[Y|X,T=1]
        self.outcome_model_1_ = Ridge(alpha=1.0)
        self.outcome_model_1_.fit(X[treated_mask], Y[treated_mask])
        self.mu_1_ = self.outcome_model_1_.predict(X)
        
        # mu_0(X) = E[Y|X,T=0]
        self.outcome_model_0_ = Ridge(alpha=1.0)
        self.outcome_model_0_.fit(X[control_mask], Y[control_mask])
        self.mu_0_ = self.outcome_model_0_.predict(X)
        
        return self
    
    def estimate_ate(self) -> Tuple[float, float]:
        """估计 ATE 和标准误"""
        if not hasattr(self, 'propensity_'):
            raise ValueError("Must fit before estimating ATE")
        
        # AIPW 得分
        aipw_scores = self._compute_aipw_scores()
        
        # ATE 估计
        ate = aipw_scores.mean()
        
        # 标准误
        se = aipw_scores.std() / np.sqrt(len(self.Y_))
        
        return ate, se
    
    def _compute_aipw_scores(self) -> np.ndarray:
        """
        计算 AIPW 得分
        
        AIPW = (mu_1 - mu_0) + T*(Y - mu_1)/e - (1-T)*(Y - mu_0)/(1-e)
        """
        # 第一项：回归调整
        term1 = self.mu_1_ - self.mu_0_
        
        # 第二项：处理组 IPW 修正
        term2 = self.T_ * (self.Y_ - self.mu_1_) / self.propensity_
        
        # 第三项：控制组 IPW 修正
        term3 = (1 - self.T_) * (self.Y_ - self.mu_0_) / (1 - self.propensity_)
        
        aipw_scores = term1 + term2 - term3
        
        return aipw_scores
    
    def decompose_components(self) -> Dict[str, float]:
        """分解 AIPW 的三个组成部分"""
        term1 = (self.mu_1_ - self.mu_0_).mean()
        term2 = (self.T_ * (self.Y_ - self.mu_1_) / self.propensity_).mean()
        term3 = ((1 - self.T_) * (self.Y_ - self.mu_0_) / (1 - self.propensity_)).mean()
        
        return {
            'regression_component': term1,
            'treated_ipw_correction': term2,
            'control_ipw_correction': -term3,
            'total_ate': term1 + term2 - term3
        }
    
    def summary(self):
        """打印估计结果摘要"""
        ate, se = self.estimate_ate()
        components = self.decompose_components()
        
        print("=" * 70)
        print("AIPW 估计结果")
        print("=" * 70)
        print(f"ATE 估计: {ate:.4f} ± {se:.4f}")
        print(f"95% CI: [{ate - 1.96*se:.4f}, {ate + 1.96*se:.4f}]")
        print()
        print("组成部分分解:")
        print(f"  1. 回归调整项:        {components['regression_component']:8.4f}")
        print(f"  2. 处理组 IPW 修正:   {components['treated_ipw_correction']:+8.4f}")
        print(f"  3. 控制组 IPW 修正:   {components['control_ipw_correction']:+8.4f}")
        print(f"  {'='*30}")
        print(f"  总 ATE:              {components['total_ate']:8.4f}")
        print("=" * 70)


# 测试我们的实现
print("🧪 测试自己实现的 AIPW 估计器\n")

# 生成数据（包含非线性）
np.random.seed(42)
n = 2000
X_test = np.random.randn(n, 3)

# 真实 DGP（带非线性）
propensity_logit = 1.5 * (X_test[:, 0] + 0.5 * X_test[:, 1] + 0.2 * X_test[:, 0]**2)
propensity_true = 1 / (1 + np.exp(-propensity_logit))
T_test = np.random.binomial(1, propensity_true)
Y_test = 2.0 * T_test + 1.5 * X_test[:, 0] + X_test[:, 1] + 0.3 * X_test[:, 0]**2 + np.random.randn(n) * 0.5

print(f"真实 ATE = 2.0")
print(f"数据特点: 包含非线性项 X1^2\n")

# 测试我们的 AIPW
my_aipw = MyAIPWEstimator()
my_aipw.fit(X_test, T_test, Y_test)
my_aipw.summary()

# 对比其他方法
print("\n\n对比其他方法:")
print("=" * 70)

# 朴素估计
naive = Y_test[T_test==1].mean() - Y_test[T_test==0].mean()
print(f"朴素估计:     {naive:.4f}  (偏差: {naive - 2.0:+.4f})")

# 纯回归调整（会因非线性而有偏）
reg_component = my_aipw.decompose_components()['regression_component']
print(f"回归调整:     {reg_component:.4f}  (偏差: {reg_component - 2.0:+.4f})")

# AIPW
aipw_ate, _ = my_aipw.estimate_ate()
print(f"AIPW:        {aipw_ate:.4f}  (偏差: {aipw_ate - 2.0:+.4f}) ✅")

print("\n💡 观察: AIPW 通过 IPW 修正项纠正了回归调整的偏差！")

---

## 💻 从零实现 AIPW

让我们实现一个完整的 AIPW 估计器类！

---

## 🎤 高频面试题

### 面试题 1: 什么是双重稳健性？为什么 AIPW 具有这个性质？

**参考回答**:

双重稳健性(Double Robustness)是指：**只要倾向得分模型或结果模型之一正确，估计量就是一致的。**

**AIPW 的公式**:
$$\hat{\tau} = \frac{1}{n}\sum_i \left[(\hat{\mu}_1(X_i) - \hat{\mu}_0(X_i)) + \frac{T_i(Y_i - \hat{\mu}_1(X_i))}{\hat{e}(X_i)} - \frac{(1-T_i)(Y_i - \hat{\mu}_0(X_i))}{1-\hat{e}(X_i)}\right]$$

**为什么具有双重稳健性**:

1. **如果倾向得分正确**: IPW 修正项会完美抵消结果模型的误差
2. **如果结果模型正确**: 残差 $Y - \hat{\mu}(X)$ 的期望为 0，IPW 修正项不引入偏差

**直观理解**: AIPW 就像买了两份保险：
- 第一份险：结果模型预测效应
- 第二份险：IPW 修正任何预测误差
- 只要一份险有效，就能得到正确答案！

---

### 面试题 2: AIPW 估计器的三项分别代表什么含义？

**参考回答**:

| 项 | 公式 | 统计含义 | 因果含义 |
|---|------|----------|----------|
| **第一项** | $\hat{\mu}_1(X) - \hat{\mu}_0(X)$ | 两个回归模型预测的差异 | 基于协变量预测的个体效应 |
| **第二项** | $\frac{T(Y - \hat{\mu}_1(X))}{e(X)}$ | 处理组的加权残差 | 修正处理组预测误差 |
| **第三项** | $-\frac{(1-T)(Y - \hat{\mu}_0(X))}{1-e(X)}$ | 控制组的加权残差 | 修正控制组预测误差 |

**工作原理**:
1. 第一项给出"初步估计"（基于结果模型）
2. 第二、三项对"观测到的样本"的预测误差进行加权修正
3. 权重 $\frac{1}{e(X)}$ 确保修正是无偏的

**特殊情况**:
- 如果 $\hat{\mu}$ 完美，第二三项 = 0，只用第一项
- 如果 $\hat{\mu}$ 很差但 $\hat{e}$ 对，第二三项会完全修正第一项的误差

---

### 面试题 3: 如果两个模型都错误，AIPW 还会有偏吗？为什么？

**参考回答**:

**是的，AIPW 仍会有偏。**

双重稳健性不是"无敌"的，它的保证是：
- **至少一个模型正确 → 无偏**
- **两个模型都错 → 一般情况下有偏**

**为什么？**

AIPW 的期望为：
$$E[\hat{\tau}_{AIPW}] = \tau + \underbrace{E\left[\frac{T(\hat{\mu}_1(X) - \mu_1(X))}{e(X)} \cdot \frac{e(X) - \hat{e}(X)}{e(X)}\right]}_{\text{两个模型误差的交互项}}$$

当两个模型都错时，交互项一般不为 0，导致偏差。

**但是**，即使两个模型都有小误差，AIPW 的偏差通常也比单独使用任一模型要小！这就是它的优势。

**实践建议**:
1. 尽量用灵活的模型（如随机森林、XGBoost）
2. 两个模型用不同的模型族（降低同时错的概率）
3. 使用交叉拟合避免过拟合偏差

---

### 面试题 4: AIPW 和普通的回归调整有什么区别？

**参考回答**:

| 特性 | 回归调整 | AIPW |
|------|---------|------|
| **公式** | $\hat{\mu}_1(X) - \hat{\mu}_0(X)$ | $(\hat{\mu}_1 - \hat{\mu}_0) + \text{IPW 修正}$ |
| **依赖** | 只依赖结果模型 | 依赖两个模型 |
| **稳健性** | 模型错了就完蛋 | 双重稳健 ✅ |
| **效率** | 通常效率较高 | 两模型都对时最优，一个错时仍合理 |
| **方差** | 较小 | 可能稍大（因为有 IPW 项） |

**关键区别**：
- 回归调整假设 $\hat{\mu}$ 正确，直接用它估计
- AIPW 说"我不完全信任 $\hat{\mu}$，用 IPW 检查和修正它"

**比喻**：
- 回归调整：完全相信导航
- AIPW：相信导航，但同时看路标确认（双保险）

---

---

## 🎉 总结

### 核心公式

AIPW 估计器：

$$\hat{\tau}_{AIPW} = \frac{1}{n}\sum_{i=1}^{n}\left[(\hat{\mu}_1(X_i) - \hat{\mu}_0(X_i)) + \frac{T_i(Y_i - \hat{\mu}_1(X_i))}{\hat{e}(X_i)} - \frac{(1-T_i)(Y_i - \hat{\mu}_0(X_i))}{1-\hat{e}(X_i)}\right]$$

### 双重稳健性

| 倾向得分 | 结果模型 | AIPW |
|---------|---------|------|
| ✅ 正确 | ✅ 正确 | ✅ 无偏 |
| ✅ 正确 | ❌ 错误 | ✅ 无偏 |
| ❌ 错误 | ✅ 正确 | ✅ 无偏 |
| ❌ 错误 | ❌ 错误 | ❌ 有偏 |

### 方法选择指南

| 方法 | 优点 | 缺点 | 适用场景 |
|-----|------|------|----------|
| PSM | 直观、可检查平衡 | 丢弃样本 | 小数据、需要可解释性 |
| IPW | 使用全部样本 | 极端权重 | 中等混淆 |
| AIPW | 双重稳健、效率高 | 计算复杂 | **推荐使用** |

### 第二章完结！

恭喜你完成了处理效应估计的所有练习！你已经掌握了因果推断的核心方法：

- ✅ 倾向得分匹配 (PSM)
- ✅ 逆概率加权 (IPW)
- ✅ 双重稳健估计 (AIPW)

下一章我们将学习**异质处理效应**——不仅估计平均效应，还要识别「谁受益最多」！

---

**「双重保险，一个正确就够。」——这就是双重稳健的智慧。**